In [36]:
question = '세로가 가로의 3 배인 직사각형이 있습니다. 이 직사각형의 가로가 9cm 일 때, 네 변의 길이의 합은 몇 cm입니까?'

### **TF-IDF**

In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('item_pool.csv')

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the vectorizer
X = vectorizer.fit_transform(df['question'])
Y = vectorizer.transform([question])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(Y, X)
cosine_sim = cosine_sim.flatten()

# 코사인 유사도를 데이터프레임에 추가
df['cosine_sim'] = cosine_sim

tfidf_df = df.sort_values(by='cosine_sim', ascending=False)
tfidf_df.head(7)

,grade,semester,chapter,question,s1,s2,s3,s4,s5,answer,difficulty,explaination,cosine_sim
11,3,2,3. 원,반지름이 7cm인 원이 있습니다. 원의 지름은 몇 cm입니까?,NaN,NaN,NaN,NaN,NaN,14cm,2,"지름은 반지름의 2배의 길이를 가지므로, 반지름은 7cm * 2 = 14cm입니다",0.348762
24,3,1,4. 곱셈,빨간색 끈의 길이는 20cm입니다. 노란색 끈의 길이는 빨간색 끈의 길이의 3배입니...,NaN,NaN,NaN,NaN,NaN,240cm,1,"(노란색 끈의 길이)=(빨간색 끈의 길이)*3=20*3=60, (파란색 끈의 길이)...",0.154910
16,3,1,4. 곱셈,사탕이 20개씩 들어 있는 통이 7통 있습니다. 사탕은 모두 몇 개일까요?,NaN,NaN,NaN,NaN,NaN,140개,2,20*7=140,0.083693
22,3,1,4. 곱셈,지석이네 농장에는 닭이 32마리 있습니다. 지석이네 농장에 있는 닭의 다리는 모두 ...,NaN,NaN,NaN,NaN,NaN,64개,1,(전체 닭의 다리 수) = (닭의 수)*(닭 한마리의 다리 수)=32*2=64(개),0.066313
29,3,1,4. 곱셈,엘리베이터에 750kg까지 탈 수 있습니다. 16kg짜리 상자 4개와 몸무게가 66...,NaN,NaN,NaN,NaN,NaN,620kg,0,"(상자 무게)=16*4=64(kg), (상자와 사람 무게)=64+66=130(kg)...",0.062845
19,3,1,4. 곱셈,책꽂이 한 칸에 책이 42권씩 4칸에 꽂혀 있습니다. 책꽂이에 꽂혀 있는 책은 모두...,NaN,NaN,NaN,NaN,NaN,168권,2,42*4=168,0.062647
17,3,1,3. 나눗셈,긴 의자 한 개에 8명이 앉을 수 있습니다. 현서네 반 학생 32명이 앉으려면 긴 ...,NaN,NaN,NaN,NaN,NaN,4개,2,32/8=4,0.062632


### **Word2Vec** ##

In [38]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


df = pd.read_csv('item_pool.csv')
questions = df['question'].tolist()

sentences = [question.split() for question in questions]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(sentence, model):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# 데이터프레임의 질문들을 벡터로 변환
df['vector'] = df['question'].apply(lambda x: get_sentence_vector(x, model))

# input question을 벡터로 변환
question_vector = get_sentence_vector(question, model).reshape(1, -1)

# 코사인 유사도 계산
df['cosine_sim'] = df['vector'].apply(lambda x: cosine_similarity([x], question_vector).flatten()[0])

# 유사도 기준으로 데이터프레임 정렬
word2vec_df = df.sort_values(by='cosine_sim', ascending=False)

word2vec_df.drop(columns=['vector'], inplace=True)

word2vec_df.head(7)

,grade,semester,chapter,question,s1,s2,s3,s4,s5,answer,difficulty,explaination,cosine_sim
11,3,2,3. 원,반지름이 7cm인 원이 있습니다. 원의 지름은 몇 cm입니까?,NaN,NaN,NaN,NaN,NaN,14cm,2,"지름은 반지름의 2배의 길이를 가지므로, 반지름은 7cm * 2 = 14cm입니다",0.556453
24,3,1,4. 곱셈,빨간색 끈의 길이는 20cm입니다. 노란색 끈의 길이는 빨간색 끈의 길이의 3배입니...,NaN,NaN,NaN,NaN,NaN,240cm,1,"(노란색 끈의 길이)=(빨간색 끈의 길이)*3=20*3=60, (파란색 끈의 길이)...",0.408936
13,3,1,1. 덧셈과 뺄셈,"어느 공장에서 어린이 문제집을 1 월에는 247 권, 2월에는 321 권을 만들었습...",NaN,NaN,NaN,NaN,NaN,568권,2,247+321= 568 (권),0.379853
2,3,1,6. 분수와 소수,가희와 찬우는 똑같은 동화책을 끝까지 읽으려고 합니다. 가희는 전체의 9/13를 읽...,2배,3배,4배,1/2배,1/4배,3,0,"가희가 더 읽어야 하는 양은 전체의 4/13이고, 찬우가 더 읽어야 하는 양은 1/...",0.363070
25,3,1,4. 곱셈,주차장에 자동차 13대와 오토바이 16대가 있습니다. 주차장에 있는 자동차의 바퀴는...,16개,17개,18개,19개,20개,5,0,"(자동차 바퀴 수)=(자동차 수)*4=13*4=52(개), (오토바이 바퀴 수)=(...",0.318944
7,3,1,1. 덧셈과 뺄셈,슬기는 아침마다 달리기를 합니다. 오늘 아침에는 247m 를 달렸습니다. 내일부터는...,NaN,NaN,NaN,NaN,NaN,784m,1,"오늘 달린 거리 : 247m ,\n1 일 후 달려야 하는 거리 : 247+179=4...",0.297434
27,3,1,3. 나눗셈,한 자루에 27개씩 담긴 무가 40자루 있습니다. 이 무를 다시 한 자루에 15개씩...,NaN,NaN,NaN,NaN,NaN,195개,0,(무의 총 개수)=(한 자루에 담긴 무의 수)*(자루의 수)=27*40=1080(개...,0.288271


### **Doc2Vec**

In [39]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

df = pd.read_csv('item_pool.csv')
questions = df['question'].tolist()

# TaggedDocument로 변환
tagged_data = [TaggedDocument(words=question.split(), tags=[i]) for i, question in enumerate(questions)]

# Doc2Vec 모델 훈련
model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4, epochs=40)

def get_doc2vec_vector(sentence, model):
    words = sentence.split()
    return model.infer_vector(words)

# 데이터프레임의 질문들을 벡터로 변환
df['vector'] = df['question'].apply(lambda x: get_doc2vec_vector(x, model))

# input question을 벡터로 변환
question_vector = get_doc2vec_vector(question, model).reshape(1, -1)

# 코사인 유사도 계산
df['cosine_sim'] = df['vector'].apply(lambda x: cosine_similarity([x], question_vector).flatten()[0])

# 유사도 기준으로 데이터프레임 정렬
doc2vec_df = df.sort_values(by='cosine_sim', ascending=False)

doc2vec_df.drop(columns=['vector'], inplace=True)

doc2vec_df.head(7)


,grade,semester,chapter,question,s1,s2,s3,s4,s5,answer,difficulty,explaination,cosine_sim
10,3,2,5. 들이와 무게,5L 800mL 들이의 빈 항아리에 물을 지우가 2L 100mL 부은 후 은주가 1...,NaN,NaN,NaN,NaN,NaN,2L 100ml,0,(지우와 은주가 항아리에 부은 물의 양)\n= 2L 100mL+1L 600mL\n=...,0.997909
27,3,1,3. 나눗셈,한 자루에 27개씩 담긴 무가 40자루 있습니다. 이 무를 다시 한 자루에 15개씩...,NaN,NaN,NaN,NaN,NaN,195개,0,(무의 총 개수)=(한 자루에 담긴 무의 수)*(자루의 수)=27*40=1080(개...,0.997784
35,3,1,3. 나눗셈,준식이네 학교에서는 크리스마스를 맞이하여 크리스마스 카드 360장을 준비하여 9개의...,60장,50장,40장,30장,NaN,3,2,크리스마스 카드 360장을 준식이네 학교 9학급에 똑같이 나누어 주 기 위해서는 3...,0.997612
30,3,1,3. 나눗셈,예란이는 친구들 39명과 함께 동시에 가위바위보 놀이를 하기로 하였습니다. 그러나 ...,NaN,NaN,NaN,NaN,NaN,"가위, 4개",0,예란이가 친구 39명과 동시에 가위바위보를 했으므로 참여한 친구는 모두 40명입니다...,0.997593
1,3,1,6. 분수와 소수,직사각형 모양의 종이를 반으로 접은 다음 다시 반으로 접었습니다. 이것을 한 번 더...,NaN,NaN,NaN,NaN,NaN,1/8,2,펼쳤을 때 똑같이 나누어진 작은 도형은 8개입니다.,0.997580
34,3,1,4. 곱셈,12월 한 달 동안 진선이는 매일 팔굽혀펴기 22회를 했다. 진선이 가 한 달 동안...,572회,582회,672회,682회,NaN,4,1,12월은 한 달이 31일입니다. 팔굽혀펴기를 매일 22개씩 했으므로 한 달 동안 한...,0.997561
0,3,1,5. 길이와 시간,"민준이는 숙제인 리코더 연습을 하고, 이어서 글씨 쓰기 연습을 했습니다. 숙제를 끝...",NaN,NaN,NaN,NaN,NaN,10시 51분 5초,1,(글씨 쓰기 연습을 시작한 시각)=11시 2분 59초-7분 38초=10시 55분 2...,0.997538


### **BERT** ##

In [40]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_vector(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    # Get the mean pooling of the token embeddings
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()

df = pd.read_csv('item_pool.csv')
questions = df['question'].tolist()

# 데이터프레임의 질문들을 벡터로 변환
df['vector'] = df['question'].apply(lambda x: get_bert_vector(x, tokenizer, model))

# input question을 벡터로 변환
question_vector = get_bert_vector(question, tokenizer, model).reshape(1, -1)

# 코사인 유사도 계산
df['cosine_sim'] = df['vector'].apply(lambda x: cosine_similarity([x], question_vector).flatten()[0])

# 유사도 기준으로 데이터프레임 정렬
bert_df = df.sort_values(by='cosine_sim', ascending=False)

bert_df.drop(columns=['vector'], inplace=True)

bert_df.head(7)

,grade,semester,chapter,question,s1,s2,s3,s4,s5,answer,difficulty,explaination,cosine_sim
17,3,1,3. 나눗셈,긴 의자 한 개에 8명이 앉을 수 있습니다. 현서네 반 학생 32명이 앉으려면 긴 ...,NaN,NaN,NaN,NaN,NaN,4개,2,32/8=4,0.980606
43,3,1,4. 곱셈,감이 한 상자에 12개씩 8상자 있습니다. 감을 한 봉지에 6개씩 담으려고 합니다....,14봉지,15봉지,16봉지,17봉지,NaN,3,1,감은 모두 12×8 = 96(개)입니다. 96÷6 =.16이므로 감을 16봉지에 담...,0.980521
20,3,1,3. 나눗셈,"우리 반은 남학생이 47명, 여학생이 16명입니다. 우리반 학생을 7모둠으로 나누었...",NaN,NaN,NaN,NaN,NaN,9명,1,"우리 반의 학생의 총 수 = 47+16 = 63(명), 7모둠으로 나누어 앉으려면 ...",0.977577
30,3,1,3. 나눗셈,예란이는 친구들 39명과 함께 동시에 가위바위보 놀이를 하기로 하였습니다. 그러나 ...,NaN,NaN,NaN,NaN,NaN,"가위, 4개",0,예란이가 친구 39명과 동시에 가위바위보를 했으므로 참여한 친구는 모두 40명입니다...,0.975160
44,3,1,3. 나눗셈,연필 282자루를 6명에게 남김없이 독같이 나누어 주려고 한다. 한 명에게 줄 연필...,45자루,46자루,47자루,48자루,NaN,3,2,282÷6=47(자루),0.973942
18,3,1,3. 나눗셈,연필 54자루를 한 명에게 6자루씩 주려고 합니다. 몇 명에게 나누어 줄 수 있을까요?,NaN,NaN,NaN,NaN,NaN,9명,2,54/6=9,0.972788
41,3,1,4. 곱셈,길이가 91 m인 도로의 양쪽에 처음부터 끝까지 7 m 간격으로 나 무를 심으려고 ...,20그루,24그루,28그루,32그루,NaN,3,0,(도로 한쪽의 나무 사이의 간격 수) ＝91÷7＝13(군데) (도로 한쪽에 심을 때...,0.972700


In [41]:
tf_list = tfidf_df['question'].head(7).index.tolist()
wv_list = word2vec_df['question'].head(7).index.tolist()
dv_list = doc2vec_df['question'].head(7).index.tolist()
bert_list = bert_df['question'].head(7).index.tolist()

print('TF-IDF', tf_list, '\n')
print('Word2Vec', wv_list, '\n')
print('Doc2Vec', dv_list, '\n')
print('BERT', bert_list)

TF-IDF [11, 24, 16, 22, 29, 19, 17] 

Word2Vec [11, 24, 13, 2, 25, 7, 27] 

Doc2Vec [10, 27, 35, 30, 1, 34, 0] 

BERT [17, 43, 20, 30, 44, 18, 41]
